# Reloading


In [38]:
import mymodule
import importlib

importlib.reload(mymodule)  # just to make sure it's like the first time


module loaded
x = 10
mymodule
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: /Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/ipykernel_launcher.py


<module 'mymodule' from '/Users/davidpetrofsky/repos/snippets/python/mymodule.py'>

# Redundant imports


When a module is loaded again, either from the same or a different file, with the same or a different name, the **cached** loaded module is used. All the **state is shared**.

All the different references to the module are references to the same memory.

When any one of them is reloaded with importlib, **all are reloaded**.

Thus, it is ok to import expensive libraries in helper files for smaller purposes (like type hints) if your helper is meant to be used in a project that uses that big library already.


In [39]:
importlib.reload(mymodule)  # loaded
print(mymodule.x)
import mymodule  # not loaded

mymodule.x = 20
import mymodule as myothermodule

print(mymodule.x)
print(myothermodule.x)  # same value!

importlib.reload(mymodule)  # loaded
print(myothermodule.x)

module loaded
x = 10
mymodule
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: /Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/ipykernel_launcher.py
10
20
20
module loaded
x = 10
mymodule
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: /Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/ipykernel_launcher.py
10


# Script vs. Module

The module I'm loading here prints 'main' when \_\_name\_\_=='\_\_main\_\_'.

When name is not \_\_main\_\_, it is the **name of the module itself**.


In [47]:
importlib.reload(mymodule)  # doesn't print 'main'


module loaded
x = 10
mymodule
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: /Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/ipykernel_launcher.py


<module 'mymodule' from '/Users/davidpetrofsky/repos/snippets/python/mymodule.py'>

In [48]:
!python3 -m mymodule # prints main even though module

module loaded
x = 10
main
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: /Users/davidpetrofsky/repos/snippets/python/mymodule.py


In [49]:
!python3 -c 'import mymodule' # does not print main

module loaded
x = 10
mymodule
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: -c


In [50]:
!python3 mymodule.py # prints main

module loaded
x = 10
main
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: mymodule.py


In [51]:
print(__name__)  # jupyter notebooks present as scripts


__main__


# File Name


Note that **\_\_file\_\_** is the module filename and **\_\_argv[0]\_\_** is who imported the module.


In [45]:
import mymodule

importlib.reload(
    mymodule)  # __file__ inside the module is always the module file


module loaded
x = 10
mymodule
file: /Users/davidpetrofsky/repos/snippets/python/mymodule.py
argv0: /Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/ipykernel_launcher.py


<module 'mymodule' from '/Users/davidpetrofsky/repos/snippets/python/mymodule.py'>

In [46]:
print('__file__' in locals())  # jupyter notebook has no __file__ member


False


In [52]:
import sys

print(sys.argv[0])  # iPython kernel in this case


/Users/davidpetrofsky/miniforge3/envs/ai/lib/python3.10/site-packages/ipykernel_launcher.py


# Other Names


In [58]:
class MyClass:

    def f():
        pass

    pass


def g():
    pass


print(MyClass.__name__)
print(MyClass.f.__name__)
print(g.__name__)

MyClass
f
g


# Paths and Imports


Full-qualified module names.


In [62]:
# submodule.py is in subfolder, relative to notebook (cwd)
import subfolder.submodule

print(subfolder.submodule.x)

10


Giving a different local name for clarity.


In [63]:
import subfolder.submodule as themodule

print(themodule.x)

10


Eliminate the need to fully qualify it.


In [64]:
from subfolder import submodule

print(submodule.x)

10


Symbols inside can be referenced this way too.

NOTE: the whole module must still be loaded for this member to be able to exist.


In [66]:
from subfolder.submodule import x

print(x)

10


In [67]:
from subfolder.submodule import x as better_variable_name

print(better_variable_name)

10


# Search Path

When you import modules by name, a search path is used to resolve the path.

You can add to the search path for easier imports of your libraries via:

- `sys.path.append()` in a .py file itself
- adding to `PYTHONPATH` variable in your environment


In [82]:
from subfolder import submodule
import os
import sys

subfolder_path = os.path.dirname(submodule.__file__)
sys.path.append(subfolder_path)

import submodule  # works because we added to search path


# Package


If you look in **packagefolder** there is an **\_\_init\_\_.py** file which is the code that runs when you import the package. There is an import inside there that looks like this: `from .deepersubfolder import deepmodule`

**.deepersubfolder** is a **package-relative import**.

Note that the deepersufolder part of the path is gone from the module name. If you need more control, you can use a **hierarchy of packages** by including more \_\_init\_\_.py files.

Another couple of things I didn't show here but you should remember:

- there is also .. for relative imports
- there is also \* for importing all symbols in a module


In [88]:
import packagefolder

print(packagefolder.deepmodule.deep_x)

100


This is **illegal** - the name deepmodule doesn't exist in packagefolder until you import that package and it doesn't automatically do it for you.

Note that this fails even though I already imported packagefolder above!


In [89]:
import packagefolder.deepmodule

ModuleNotFoundError: No module named 'packagefolder.deepmodule'

# Unexpected Name Clash Issue


- if you have a file called `mymodule.py` and that file has the line `import mymodule`, it is trying to import itself
- this seems obvious in this example, but it can happen to you unexpectedly, and no helpful info is given
  - the symptom would be that there is nothing in the module
  - eg. I made a file called openai.py in my library to call openai's API, which has that same name
- another place this comes up is if you have a folder named after you python script
  - eg. _mymodule folder_ in same folder as mymodule.py
  - then you try to `import mymodule.someothermodule`
    - because mymodule.py exists, this doesn't work


# Terminology: Wheels


- a wheel is like a compiled binary version of a package
- even though the packages are python, they often have compiled C++ portions (as tensorflow does)
- those portions must be compiled for the target system
- when you pip install, you get the precompiled wheels
- sometimes (mostly on apple silicon), you either can't pip install something or it fails due to not being properly compiled for that architecture
- `pip install --no-binary :all:` is a trick to make the binary build on your system which might sometimes resolve it (and sometimes not)
- the apple people are very active on conda-forge, so conda install often has much better wheels for apple in particular
- the apple channel (as opposed to conda-forge) also has stuff from the apple team specifically for that purpose pip kind of sees (some of) what conda has installed, and conda sees (some of) what pip installs, but they're not equivalent
- for instance, they can't properly uninstall each other's stuff
- dependency resolution in both systems doesn't tend to know that you had an old version because it had a better wheel for M1


# Disallowed File/Module Names

-  dash is not allowed (because not allowed in identifiers)
-  . is not explicitly disallowed but is ambiguous with folder and thus won't work

# \_\_main\_\_.py

`__main.py__`, if present, is the file that will run first when a folder is run from the commandline.

It works with or without the `-m` flag for using a module name instead of path.

If you use this mechanism, you don't need to use the `__name__ == '__main__'` idiom in that file and probably don't need it anymore in `__init__.py` (for importing as a module) either.

In [4]:
!!python3 myapp

['hi, this is my app']

In [5]:
!!python3 -m myapp

['hi, this is my app']